In [16]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import warnings 
import optuna
import joblib


warnings.filterwarnings('ignore')

In [5]:
os.chdir("c://Users//divya//desktop//TCS_TASK-1")

In [7]:
from src.pipeline import merge_and_melt, fill_sell, fill_event, cat_to_num, add_lags, add_mean_encoding, add_sold_mean, fill_df
from src.utils import load_data

In [8]:
def prepare_dataframe():
    """Runs the full data pipeline."""
    sales_df, calendar_df, prices_df, _ = load_data()
    
    print("--- Merging and Melting Data ---")
    data_df = merge_and_melt(sales_df, calendar_df, prices_df)
    
    print("--- Filling Nulls and Converting Categories ---")
    data_df = fill_sell(data_df)
    data_df = fill_event(data_df)
    data_df = cat_to_num(data_df)
    
    print("--- Creating Features ---")
    lags = [1, 2, 3, 7, 15, 30]
    data_df = add_lags(data_df, lags)
    data_df = add_mean_encoding(data_df)
    data_df = add_sold_mean(data_df)
    
    print("--- Finalizing Dataset ---")
    data_df = fill_df(data_df)
    
    return data_df

In [9]:
data = prepare_dataframe()

--- Merging and Melting Data ---
--- Filling Nulls and Converting Categories ---


c:\Users\divya\desktop\TCS_TASK-1\src\pipeline.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['sell_price'] = df.groupby(['store_id', 'item_id'])['sell_price'].bfill()


--- Creating Features ---
--- Finalizing Dataset ---


In [14]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,...,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg,rolling_sold_mean,expanding_sold_mean,selling_trend
0,14370,1437,3,1,0,0,31,0,11105,1,...,0.703125,0.313721,0.213989,0.213989,1.320312,0.811035,1.032227,0.0,0.0,-0.313721
1,14380,1438,3,1,0,0,31,0,11105,1,...,0.703125,0.257812,0.264404,0.264404,1.320312,0.811035,1.032227,0.0,0.0,-0.257812
2,14390,1439,3,1,0,0,31,0,11105,1,...,0.703125,0.150513,0.075012,0.075012,1.320312,0.811035,1.032227,0.0,0.0,-0.150513
3,14400,1440,3,1,0,0,31,0,11105,1,...,0.703125,1.718750,2.046875,2.046875,1.320312,0.811035,1.032227,0.0,0.0,-1.718750
4,14410,1441,3,1,0,0,31,0,11105,1,...,0.703125,0.966309,0.764160,0.764160,1.320312,0.811035,1.032227,0.0,0.0,-0.966309


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57412670 entries, 0 to 57412669
Data columns (total 42 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int16  
 1   item_id                   int16  
 2   dept_id                   int8   
 3   cat_id                    int8   
 4   store_id                  int8   
 5   state_id                  int8   
 6   d                         int16  
 7   sold                      int16  
 8   wm_yr_wk                  int16  
 9   weekday                   int8   
 10  wday                      int8   
 11  month                     int8   
 12  year                      int16  
 13  event_name_1              int8   
 14  event_type_1              int8   
 15  event_name_2              int8   
 16  event_type_2              int8   
 17  snap_CA                   int8   
 18  snap_TX                   int8   
 19  snap_WI                   int8   
 20  sell_price            

In [ ]:
def optimize_and_evaluate_models(data):
    """
    Trains and evaluates models for each store using a validation set.
    This is for finding the best parameters and validating performance locally.
    """
    print("--- Starting Local Evaluation Phase ---")
    
    valid_indices = data[(data['d'] >= 1885) & (data['d'] < 1914)].index
    valid_preds = pd.Series(index=valid_indices, dtype=float, name='validation_predictions')
    
    stores = data.store_id.unique().tolist()

    for store in stores:
        df = data[data['store_id'] == store]
        
        X_train, y_train = df[df['d'] < 1885].drop('sold', axis=1), df[df['d'] < 1885]['sold']
        X_valid, y_valid = df[(df['d'] >= 1885) & (df['d'] < 1914)].drop('sold', axis=1), df[(df['d'] >= 1885) & (df['d'] < 1914)]['sold']
        
        model = LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            max_depth=8,
            num_leaves=50,
            min_child_weight=100,
            verbose=-1
        )
        print(f'----- Optimizing for Store: {store} -----')
        
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse',
                  callbacks=[lgb.early_stopping(20)])
        
        valid_predictions = model.predict(X_valid)
        rmse = np.sqrt(mean_squared_error(y_valid, valid_predictions))
        print(f'Validation RMSE for {store}: {rmse:.4f}')
        valid_preds.loc[X_valid.index] = valid_predictions

    return valid_preds

In [ ]:
#def optimize_and_evaluate_models(data):
"""
Trains and evaluates models for each store using a validation set.
"""
print("--- Starting Local Evaluation Phase ---")
d_store_id = {0: 'CA_1', 1: 'CA_2', 2: 'CA_3', 3: 'CA_4', 4: 'TX_1', 5: 'TX_2', 6: 'TX_3', 7: 'WI_1', 8: 'WI_2', 9: 'WI_3'}
stores = data.store_id.unique().tolist()

if not os.path.exists('lgbmodels'):
    os.makedirs('lgbmodels')
    print("Created directory: lgbmodels")


for store in stores:
    df = data[data['store_id'] == store]
    
    X_train, y_train = df[df['d'] < 1885].drop('sold', axis=1), df[df['d'] < 1885]['sold']
    X_valid, y_valid = df[(df['d'] >= 1885) & (df['d'] < 1914)].drop('sold', axis=1), df[(df['d'] >= 1885) & (df['d'] < 1914)]['sold']
    
    def objective(trial):
        params = {
            'objective': 'regression_l1',
            'metric': 'rmse',
            'n_estimators': 700,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'max_depth': trial.suggest_int('max_depth', 5, 12),
            'num_leaves': trial.suggest_int('num_leaves', 30, 150),
            'min_child_weight': trial.suggest_int('min_child_weight', 50, 500),
            'verbose': -1,
            'n_jobs': -1,
            'random_state': 42
        }
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse', callbacks=[lgb.early_stopping(20, verbose=False)])
        return model.best_score_['valid_0']['rmse']
    print(f'----- Optimizing for Store: {d_store_id[store]} -----')
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=8, show_progress_bar=True)
    
    best_params = study.best_params
    best_rmse = study.best_value
    print(f"Tuning finished. Best RMSE for {d_store_id[store]}: {best_rmse:.4f}")
    joblib.dump(best_params, f'lgbmodels/best_params_{d_store_id[store]}.pkl')

--- Starting Local Evaluation Phase ---
Created directory: lgbmodels


[I 2025-09-14 06:42:15,841] A new study created in memory with name: no-name-89d66e6d-1deb-4860-8e3f-e041eb7846cc


----- Optimizing for Store: CA_1 -----


Best trial: 0. Best value: 0.576765:  12%|█▎        | 1/8 [02:30<17:31, 150.14s/it]

[I 2025-09-14 06:44:45,984] Trial 0 finished with value: 0.5767646366174688 and parameters: {'learning_rate': 0.15289188828326689, 'subsample': 0.7809394060781171, 'colsample_bytree': 0.9153035969081716, 'max_depth': 7, 'num_leaves': 113, 'min_child_weight': 352}. Best is trial 0 with value: 0.5767646366174688.


...

[I 2025-09-14 10:25:44,072] Trial 5 finished with value: 0.6283111249560935 and parameters: {'learning_rate': 0.19884003269103204, 'subsample': 0.9418934172261277, 'colsample_bytree': 0.748848702465295, 'max_depth': 7, 'num_leaves': 82, 'min_child_weight': 480}. Best is trial 5 with value: 0.6283111249560935.


Best trial: 5. Best value: 0.628311:  88%|████████▊ | 7/8 [18:07<02:42, 162.74s/it]

[I 2025-09-14 10:28:36,590] Trial 6 finished with value: 0.6477617588283114 and parameters: {'learning_rate': 0.10277690993568782, 'subsample': 0.8815334273039608, 'colsample_bytree': 0.8674682747126156, 'max_depth': 12, 'num_leaves': 84, 'min_child_weight': 305}. Best is trial 5 with value: 0.6283111249560935.


Best trial: 5. Best value: 0.628311: 100%|██████████| 8/8 [20:24<00:00, 153.03s/it]

[I 2025-09-14 10:30:53,221] Trial 7 finished with value: 0.680667739515964 and parameters: {'learning_rate': 0.10928985182480255, 'subsample': 0.8017756917369361, 'colsample_bytree': 0.6307793926500257, 'max_depth': 6, 'num_leaves': 115, 'min_child_weight': 288}. Best is trial 5 with value: 0.6283111249560935.
Tuning finished. Best RMSE for WI_3: 0.6283


In [25]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

print("--- Starting Local Evaluation Phase ---")
d_store_id = {0: 'CA_1', 1: 'CA_2', 2: 'CA_3', 3: 'CA_4', 4: 'TX_1', 5: 'TX_2', 6: 'TX_3', 7: 'WI_1', 8: 'WI_2', 9: 'WI_3'}
stores = data.store_id.unique().tolist()

if not os.path.exists('xgbmodels'):
    os.makedirs('xgbmodels')
    print("Created directory: xgbmodels")

for store in stores:
    df = data[data['store_id'] == store]
    
    X_train, y_train = df[df['d'] < 1885].drop('sold', axis=1), df[df['d'] < 1885]['sold']
    X_valid, y_valid = df[(df['d'] >= 1885) & (df['d'] < 1914)].drop('sold', axis=1), df[(df['d'] >= 1885) & (df['d'] < 1914)]['sold']
    
    def objective(trial):
        params = {
            'objective': 'reg:squarederror', 
            'n_estimators': 700,
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'max_depth': trial.suggest_int('max_depth', 5, 12),
            'min_child_weight': trial.suggest_int('min_child_weight', 50, 500),
            'verbosity': 0,
            'n_jobs': -1,
            'random_state': 42,
            'early_stopping_rounds': 20,
            'eval_metric': 'rmse' 
        }
        
        model = xgb.XGBRegressor(**params)
        
        model.fit(
            X_train, y_train, 
            eval_set=[(X_valid, y_valid)],
            verbose = False
        )
        
        valid_predictions = model.predict(X_valid)
        rmse = np.sqrt(mean_squared_error(y_valid, valid_predictions))
        
        return rmse

    print(f'----- Optimizing for Store: {d_store_id[store]} -----')
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=8, show_progress_bar=True)
    
    best_params = study.best_params
    best_rmse = study.best_value
    print(f"Tuning finished. Best RMSE for {d_store_id[store]}: {best_rmse:.4f}")
    joblib.dump(best_params, f'xgbmodels/best_params_{d_store_id[store]}.pkl')

--- Starting Local Evaluation Phase ---


[I 2025-09-14 14:07:30,818] A new study created in memory with name: no-name-7a4a0d1e-5241-4398-9777-3eae0a91b4f6


----- Optimizing for Store: CA_1 -----


Best trial: 0. Best value: 0.551531:  12%|█▎        | 1/8 [00:59<06:57, 59.69s/it]

[I 2025-09-14 14:08:30,520] Trial 0 finished with value: 0.5515309635472638 and parameters: {'learning_rate': 0.06286781614138935, 'subsample': 0.8926341294025253, 'colsample_bytree': 0.7337378421256954, 'max_depth': 12, 'min_child_weight': 107}. Best is trial 0 with value: 0.5515309635472638.


...

[I 2025-09-14 15:38:49,168] Trial 5 finished with value: 0.5776255486408758 and parameters: {'learning_rate': 0.13627530430438628, 'subsample': 0.6129987295360498, 'colsample_bytree': 0.9128218781988009, 'max_depth': 12, 'min_child_weight': 77}. Best is trial 5 with value: 0.5776255486408758.


Best trial: 5. Best value: 0.577626:  88%|████████▊ | 7/8 [07:50<00:52, 52.34s/it]

[I 2025-09-14 15:39:35,974] Trial 6 finished with value: 0.6467186771161447 and parameters: {'learning_rate': 0.042994000171791606, 'subsample': 0.9321313164838317, 'colsample_bytree': 0.9762555963588679, 'max_depth': 5, 'min_child_weight': 71}. Best is trial 5 with value: 0.5776255486408758.


Best trial: 5. Best value: 0.577626: 100%|██████████| 8/8 [09:22<00:00, 70.35s/it]

[I 2025-09-14 15:41:07,825] Trial 7 finished with value: 0.5901086477685822 and parameters: {'learning_rate': 0.025425407012468926, 'subsample': 0.874635035386998, 'colsample_bytree': 0.7940290088457628, 'max_depth': 8, 'min_child_weight': 182}. Best is trial 5 with value: 0.5776255486408758.
Tuning finished. Best RMSE for WI_3: 0.5776
